# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#VactionPy/VacationPy.ipynb
cities_df = pd.read_csv("output_data/cities.csv")
cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,0,nemuro,43.3236,145.5750,57.47,86,100,25.48,JP,1628631012
1,1,tilichiki,60.4667,166.1000,59.81,54,88,4.14,RU,1628631035
2,2,mataura,-46.1927,168.8643,43.48,94,27,3.40,NZ,1628631553
3,3,codrington,-38.2667,141.9667,48.94,81,100,15.79,AU,1628631567
4,4,santiago del estero,-27.7951,-64.2615,57.81,41,0,5.73,AR,1628631568
...,...,...,...,...,...,...,...,...,...,...
446,446,garzon,2.1959,-75.6278,75.49,84,34,4.63,CO,1628631705
447,447,luderitz,-26.6481,15.1594,61.86,53,29,1.61,NaN,1628631706
448,448,havoysund,70.9963,24.6622,63.48,83,100,8.16,NO,1628631706
449,449,macau,22.2006,113.5461,85.06,89,40,13.80,MO,1628631706


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
#Set variables for Lat/Lng
locations = cities_df[['Lat','Lng']]

#Set a variable for humidity
humidity = cities_df['Humidity']

#Create heat map 
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

# Add the layer to the map
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Create DF for perfect weather categories. 
perfect_vaca_weather = cities_df.loc[cities_df['Humidity'] < 30, :]
perfect_vaca_weather = perfect_vaca_weather.loc[cities_df['Max Temp'] <= 89, :]
perfect_vaca_weather = perfect_vaca_weather.loc[cities_df['Max Temp'] >= 75, :]
perfect_vaca_weather = perfect_vaca_weather.loc[cities_df['Cloudiness'] <10, :]
perfect_vaca_weather = perfect_vaca_weather.loc[cities_df['Windspeed'] <20, :].dropna()
perfect_vaca_weather.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
67,67,tahta,26.7693,31.5021,88.77,25,0,6.82,EG,1628631092
157,157,tshikapa,-6.4167,20.8000,76.26,26,1,2.59,CD,1628631612
191,191,cedar city,37.6775,-113.0619,88.18,14,1,5.75,US,1628631623
303,303,elko,41.0002,-115.5012,88.65,3,0,1.01,US,1628631657


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Create Hotel DF and add Hotel Name into DF
hotel_df = perfect_vaca_weather[['City', 'Country', 'Lat', 'Lng', 'Max Temp']].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Max Temp,Hotel Name
67,tahta,EG,26.7693,31.5021,88.77,
157,tshikapa,CD,-6.4167,20.8000,76.26,
191,cedar city,US,37.6775,-113.0619,88.18,
303,elko,US,41.0002,-115.5012,88.65,


In [7]:
# #Set url for search
# url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# #Set parameters to search for hotels with 5000 meteres & Parameters
target_search = "Hotel"
radius = 5000

# #Create parameter dict.
params = {
    "location": locations,
    "types": target_search,
    "radius": radius,
    "key": g_key
}

#Run Request
idx = 0
for index, row in hotel_df.iterrows():
    # set up a parameters dictionary
    params = {
        "location": f"{row['Lat']}, {row['Lng']}",
        "keyword": target_search,
        "radius": radius,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    response_json = response.json()
    results = response_json['results']
    if len(results) > 0:
        name = response_json['results'][0]['name']
        hotel_df.iloc[idx, -1] = name
    idx = idx + 1

hotel_results = hotel_df
hotel_results

,City,Country,Lat,Lng,Max Temp,Hotel Name
67,tahta,EG,26.7693,31.5021,88.77,El Salam Hotel
157,tshikapa,CD,-6.4167,20.8000,76.26,Guest house Armeline
191,cedar city,US,37.6775,-113.0619,88.18,Best Western Plus Cedar City
303,elko,US,41.0002,-115.5012,88.65,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# marker_locations = locations[['Lat', 'Lng']]
# Add marker layer ontop of heat map
# fig = gmaps.figure()
markers = gmaps.marker_layer(locations,
    info_box_content= hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))